In [21]:
import pandas as pd
from scipy import stats

In [22]:
df = pd.read_csv('datasets/Ashopping.csv', sep=',', encoding='CP949')
df.head()

,고객ID,이탈여부,총_매출액,방문빈도,1회_평균매출액,할인권_사용 횟수,총_할인_금액,고객등급,구매유형,클레임접수여부,...,매장_청결성,공간_편의성,시야_확보성,음향_적절성,안내_표지판_설명,친절성,신속성,책임성,정확성,전문성
0,1,0,4007080,17,235711,1,5445,1,4,0,...,6,7,6,6,6,6,6,6,6,6
1,2,1,3168400,14,226314,22,350995,2,4,0,...,7,7,6,6,6,5,3,6,6,6
2,3,0,2680780,18,148932,6,186045,1,4,1,...,6,6,6,6,6,7,7,6,6,7
3,4,0,5946600,17,349800,1,5195,1,4,1,...,6,6,5,6,6,6,6,6,5,6
4,5,0,13745950,73,188301,9,246350,1,2,0,...,5,6,6,6,5,5,6,6,5,6


In [23]:
df1=df[['총_매출액','방문빈도','구매_카테고리_수']]
df1.head()

,총_매출액,방문빈도,구매_카테고리_수
0,4007080,17,6
1,3168400,14,4
2,2680780,18,6
3,5946600,17,5
4,13745950,73,6


In [24]:
#수치형 변수 상관관계 분석 (피어슨)
print(stats.pearsonr(df1.총_매출액, df1.방문빈도))
print(stats.pearsonr(df1.총_매출액, df1.구매_카테고리_수))
print(stats.pearsonr(df1.방문빈도, df1.구매_카테고리_수))

(0.6311706453193395, 3.051960449687332e-112)
(0.30834691323967117, 1.8121422756812584e-23)
(0.39586072218464485, 7.345432750388936e-39)


상관계수와 유의확률(p-value)가 출력된다

In [25]:
df1.corr(method='pearson')

,총_매출액,방문빈도,구매_카테고리_수
총_매출액,1.000000,0.631171,0.308347
방문빈도,0.631171,1.000000,0.395861
구매_카테고리_수,0.308347,0.395861,1.000000


총매출액과 방문빈도, 총매출액과 구매 카테고리수 간의 상관관계가 유의확률 0.0001이하로 유의하지만  
매출액과 방문빈도간의 상관관계가 더 높다(0.63>0.31)  
매출액 증대를 위해서는 방문빈도를 유도하는 것이 더 필요하다.

In [26]:
#편(부분)상관관계분석 : 제 3의 변수를 통제하거나 효과를 제거한 후 특정 두 변수의 순수한 상관관계를 파악하기 위한 분석기법  
#H0(귀무가설):구매카테고리수가 통제된 상황에서 총 매출액과 방문빈도 간에는 유의한 선형 관계를 갖지 않는다.
#H1(대립가설):구매카테고리수가 통제된 상황에서 총 매출액과 방문빈도 간에는 유의한 선형 관계를 갖는다.

In [27]:
from pingouin import partial_corr  #pip install pingouin
partial_corr(data=df1, x='총_매출액', y='방문빈도', covar='구매_카테고리_수')

,n,r,CI95%,p-val
pearson,1000,0.582794,"[0.54, 0.62]",6.223289e-92


총매출액과 방문빈도간의 상관계수는 0.583이며, 유의확률 0.001이하로 도출되었다.  
구매카테고리 수를 통제하지 않은 상태에서의 상관계수인 0.631보다는 약해졌다.  
따라서, 구매카테고리가 총매출액과의 직접적인 상관관계가 크진 않지만,  
총매출액과 방문빈도에 영향을 미치므로 마케팅 전략 수립시 함께 고민해야 한다.

In [32]:
#순서형 변수 상관관계 분석 (스피어만)
df2=df[['1회_평균매출액','방문빈도']]
stats.spearmanr(df2['1회_평균매출액'],df2['방문빈도'])

SpearmanrResult(correlation=-0.4988411248473936, pvalue=4.929293870381245e-64)

상관관계는 -0.498, p-value(유의확률)은 0.001이하로 확인되었다.   
즉, 방문빈도가 높은 고객이 1회 평균 구매액 수준에서는 더 떨어진다.  

#정준상관분석  
H0(귀무가설) = 제품 만족도는 매장 만족도와 연관성이 없다.  
H1(대립가설) = 제품 만족도는 매장 만족도와 상관관계가 있다.

In [35]:
from sklearn.cross_decomposition import CCA
from scipy import stats
import numpy as np
import pandas as pd

In [38]:
#정준분석
df=pd.read_csv('datasets/CCA.csv', sep=',', encoding='CP949')
df.tail()

,품질,가격,디자인,직원 서비스,매장 시설,고객관리
9,21,13,91,79,4,75
10,35,12,71,39,2,13
11,21,73,54,54,3,36
12,10,57,20,60,5,60
13,20,66,54,63,5,46


In [41]:
U=df[['품질','가격','디자인']]
V=df[['직원 서비스','매장 시설','고객관리']]

In [45]:
cca=CCA(n_components=1).fit(U,V)
U_c, V_c = cca.transform(U,V)
U_c1 = pd.DataFrame(U_c)[0]
V_c1 = pd.DataFrame(V_c)[0]
print(U_c)
print('\n',V_c)

[[-0.55451512]
 [-0.12301628]
 [ 0.40708859]
 [ 1.4861872 ]
 [ 1.8393597 ]
 [-0.18638005]
 [-0.47604906]
 [-1.10459412]
 [-0.72231536]
 [ 0.02423555]
 [-0.69744765]
 [ 0.54958654]
 [-0.8098459 ]
 [ 0.36770595]]

 [[-0.29828858]
 [ 0.51338449]
 [ 0.95000731]
 [ 1.76514223]
 [ 1.81062484]
 [-0.74632557]
 [-1.67676657]
 [-1.00578133]
 [-1.40361614]
 [ 1.38766197]
 [-1.28827534]
 [-0.3120564 ]
 [ 0.23113188]
 [ 0.07315722]]


In [51]:
CC1=stats.pearsonr(U_c1, V_c1)
print('제1정준상관계수:',CC1)

제1정준상관계수: (0.7717687943183484, 0.0012226670192390231)


In [55]:
print('제품만족도 정준변수와 해당변수들간 정준적재량:', np.corrcoef(U_c1.T, U.T)[0,1:])
print('제품만족도 정준변수와 매장만족도 변수들간 교차적재량:', np.corrcoef(U_c1.T, V.T)[0,1:])
print('매장만족도 정준변수와 해당변수들간 정준적재량:', np.corrcoef(V_c1.T, V.T)[0,1:])
print('매장만족도 정준변수와 제품만족도 변수들간 교차적재량:', np.corrcoef(V_c1.T, U.T)[0,1:])

제품만족도 정준변수와 해당변수들간 정준적재량: [0.35045604 0.77461847 0.55191153]
제품만족도 정준변수와 매장만족도 변수들간 교차적재량: [ 0.70598452 -0.0438384   0.5889048 ]
매장만족도 정준변수와 해당변수들간 정준적재량: [ 0.91476168 -0.0568025   0.76305858]
매장만족도 정준변수와 제품만족도 변수들간 교차적재량: [0.27044132 0.5978383  0.42592516]


제품만족도='품질','가격','디자인'  
매장만족도='직원 서비스','매장 시설','고객관리'

제1정준상관계수는 0.77, P-value는 0.001로 유의미하게 강한 양의 상관관계가 있음을 확인할 수 있다.  
정준적재량을 통해 각 정준변수의 구성관계를 살펴보면,  
제품만족도의 경우, 가격>디자인>품질 순서로 중요한 역할을 하고 있으며,   
매장만족도의 경우, 직원서비스>고객관리>매장시설 순서로 중요한 역할을 하고 있다.  
교차적재량을 통해 정준변수 상호간의 상세 영향요인을 살펴보면,  
제품만족도는 직원서비스>고객관리>매장시설 순서로 중요한 역할을 하고 있고,  
매장만족도의 경우, 가격>디자인>품질 순서로 중요한 역할을 하고 있다.  
제품만족도를 위해 품질보다는 가격과 디자인, 매장만족도를 위해 직원서비스, 고객관리의 교육이 필요하다.

In [58]:
print(np.corrcoef(U_c1.T, U.T))
print('\n',np.corrcoef(U_c1.T, V.T))
print('\n',np.corrcoef(V_c1.T, V.T))
print('\n',np.corrcoef(V_c1.T, U.T))

[[ 1.          0.35045604  0.77461847  0.55191153]
 [ 0.35045604  1.          0.1748073   0.57139455]
 [ 0.77461847  0.1748073   1.         -0.06945475]
 [ 0.55191153  0.57139455 -0.06945475  1.        ]]

 [[ 1.          0.70598452 -0.0438384   0.5889048 ]
 [ 0.70598452  1.          0.05844147  0.48474289]
 [-0.0438384   0.05844147  1.          0.17046241]
 [ 0.5889048   0.48474289  0.17046241  1.        ]]

 [[ 1.          0.91476168 -0.0568025   0.76305858]
 [ 0.91476168  1.          0.05844147  0.48474289]
 [-0.0568025   0.05844147  1.          0.17046241]
 [ 0.76305858  0.48474289  0.17046241  1.        ]]

 [[ 1.          0.27044132  0.5978383   0.42592516]
 [ 0.27044132  1.          0.1748073   0.57139455]
 [ 0.5978383   0.1748073   1.         -0.06945475]
 [ 0.42592516  0.57139455 -0.06945475  1.        ]]
